In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Importing necessary packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
pd.options.display.max_colwidth = 100



In [73]:
ncbi3=pd.read_csv('data/NCBI/ncbi_full.csv', encoding='latin1', index_col=0)
ncbi3.head(5)
len(ncbi3)

,pmid,journal,title,abstract,data
201,28868222.0,3 Biotech,Linkage disequilibrium based association mapping of micronutrients in common bean (Phaseolus vul...,"Micronutrient deficiencies are of major concern in human health and plant metabolism. Iron (Fe),...",Linkage disequilibrium based association mapping of micronutrients in common bean (Phaseolus vul...
202,28868221.0,3 Biotech,The persistence and performance of phosphate-solubilizing Gluconacetobacter liquefaciens qzr14 i...,The persistence and performance of plant growth-promoting microorganisms (PGPMs) in soil are con...,The persistence and performance of phosphate-solubilizing Gluconacetobacter liquefaciens qzr14 i...
203,28868220.0,3 Biotech,Evaluation of stress effects of copper oxide nanoparticles in Brassica napus L. seedlings.,Rapid growth of nanotechnology has enabled the production and use of engineered nanoparticles (E...,Evaluation of stress effects of copper oxide nanoparticles in Brassica napus L. seedlings.. Rapi...
204,28868214.0,3 Biotech,Allelic variation at high-molecular weight and low-molecular weight glutenin subunit genes in Mo...,Glutenin is a major protein fraction contributing to the functional properties of gluten and dou...,Allelic variation at high-molecular weight and low-molecular weight glutenin subunit genes in Mo...
205,28828287.0,3 Biotech,Concurrent production and relative quantification of vasicinone from in vivo and in vitro plant ...,The present study documents a simultaneous production and comparative assessment of extracted va...,Concurrent production and relative quantification of vasicinone from in vivo and in vitro plant ...


266392

In [74]:
dframe=ncbi3['journal'].value_counts(ascending=False).reset_index()
dframe.head(10)

,index,journal
0,Scientific reports,19294
1,PloS one,15940
2,Oncotarget,8228
3,Nature communications,2843
4,Medicine,2312
5,Oncology letters,2292
6,"Sensors (Basel, Switzerland)",2162
7,International journal of molecular sciences,1967
8,Proceedings of the National Academy of Sciences of the United States of America,1945
9,BMJ open,1860


I exclude prolific and non-specific journals that publish papers  from a wide range of research areas. I also exclude uncommon journals (<300 publications over the year).

In [80]:
df=ncbi3['journal'].value_counts().reset_index()
print ("Number of journals considered is", len(df[(df['journal']>300)&(df['journal']<2500)]))

Number of journals considered is 105


In [81]:
df2=df[(df['journal']>300)&(df['journal']<2500)]
ncbi4=ncbi3[ncbi3['journal'].isin(df2['index'].values.tolist())] 

In [82]:
ncbi4=ncbi4.copy(True)

I build a model that classifies any journal paper into the 105 categories. 

1. Stemmer: I use Lancaster stemmer to reduce words to their 'stem' (core meaning based unit). It is much faster even though it's much more aggressive than porter.
2. I use TFIDF Vectorizer to find a numeric representation of the text. Term frequency (TF)counts the number of times a term occurs in a text, keywords tend to be repeated the text. Inverse document frequency (IDF) accounts for common words aka linguistic glue that occur repeatedly in multiple documents but don't contribute to identifying a class. 
3. I use bigrams, i.e. words that two occur together multiple times are taken to be a single token
4. For classification, I use SVM with a linear kernel (less computationally intensive, good for text classification with many features). My classes are imbalanced with the number of papers/journal ranging from 300 to >2000. I use class_weights = 'balanced' to balance the classes.
5. I use CalibratedClassifierCV to find probabilities of classes for the test data. I use LinearSVC instead of SVC with linear kernel because LinearSVC is much faster.
6. I use a label encoder to transform non-numerical labels to numericals.

In [87]:
# import operator
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder as LE
from sklearn.calibration import CalibratedClassifierCV
import random
import nltk
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer 

stemmer = LancasterStemmer()
def clean_words(line):
    word_tokens = nltk.word_tokenize(line)
    stemmed = [stemmer.stem(w) for w in word_tokens]
    return ' '.join(stemmed)

In [88]:
le = LE()
tfv = TfidfVectorizer(min_df=3, stop_words='english', ngram_range=(1,2))

input_data=ncbi4['data']
X = []
for data in input_data:
    X.append(clean_words(data))
X2=X
# X=X2

In [112]:
tfv.fit(X)
le.fit(ncbi4['journal'])

X = tfv.transform(X)
y = le.transform(ncbi4['journal'])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42) #stratify=y

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

LabelEncoder()

In [113]:
linear_svc = LinearSVC(class_weight='balanced', C=1)
model = CalibratedClassifierCV(linear_svc,
                                        method='sigmoid',  
                                        cv=3) 
model.fit(X_train, y_train)
print("SVC:", model.score(X_test, y_test)) #62% at 500-3000

CalibratedClassifierCV(base_estimator=LinearSVC(C=1, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
            cv=3, method='sigmoid')

SVC: 0.563643898059


In [83]:
Y_pred_SVC=model.predict (X_test)

In [3]:
import pickle
filename = 'model_300to2500.sav'
filename_tfv = 'tfv_300to2500.sav'
filename_le = 'le_300to2500.sav'

# save the models to disk

# pickle.dump(model, open(filename, 'wb'))
# pickle.dump(tfv, open(filename_tfv, 'wb'))
# pickle.dump(le, open(filename_le, 'wb'))

In [4]:
# load models from disk
loaded_model = pickle.load(open(filename, 'rb'))
loaded_tfv = pickle.load(open(filename_tfv, 'rb'))
loaded_le = pickle.load(open(filename_le, 'rb'))
result = loaded_model.score(X_test, y_test)
print("Checking accuracy of the loaded model...", result)

NameError: name 'X_test' is not defined

The accuracy of the model is 56 % i.e., my model is able to pick the right journal from 105 journals 56% of the time!

Next, I look at the posterior probabilities that a paper belongs to a particular journal. 

In [116]:
proba=model.predict_proba(X_test)
# prob_pos = model.decision_function(X_test)
# proba = (prob_pos - prob_pos.min()) / (prob_pos.max() - prob_pos.min())

In [126]:
dfcols = ['n1','n1_n', 'p1', 'n2','n2_n', 'p2','n3','n3_n', 'p3',  ]
df_max = pd.DataFrame(columns=dfcols)
dfcols2 = ['num', 'labels']
df_labels= pd.DataFrame(columns=dfcols2)

In [5]:
labels=list(le.classes_)

NameError: name 'le' is not defined

In [128]:
num=list(range(0,len(labels)))


In [129]:
df_labels['num']=num
df_labels['labels']=list(le.classes_)
dict_l=dict(zip(num,labels))

In [130]:
import heapq
vals=[]
nvals=[]
for i in proba:
    
#     vals.append(heapq.nlargest(3,i))
    nvals.append(heapq.nlargest(3, enumerate(i), key=lambda x: x[1]))

In [131]:
one=[]
two=[]
three=[]
for i in nvals:
    three.append(i[2])
    two.append(i[1])
    one.append(i[0])    

In [132]:
n1, p1 = zip(*one)
n2, p2 = zip(*two)
n3, p3 = zip(*three)

In [133]:
df_max['n1']=n1
df_max['n2']=n2
df_max['n3']=n3
df_max['p1']=p1
df_max['p2']=p2
df_max['p3']=p3


In [134]:
df_max['n1_n']=df_max['n1'].map (dict_l)
df_max['n2_n']=df_max['n2'].map (dict_l)
df_max['n3_n']=df_max['n3'].map (dict_l)

In [135]:
df_max.head(20)

,n1,n1_n,p1,n2,n2_n,p2,n3,n3_n,p3
0,93,"Sensors (Basel, Switzerland)",0.764539,40,Frontiers in microbiology,0.045965,6,BMC bioinformatics,0.017823
1,52,International journal of molecular sciences,0.218573,78,Oncogene,0.173305,58,Journal of cellular and molecular medicine,0.167615
2,93,"Sensors (Basel, Switzerland)",0.776981,18,BioMed research international,0.019079,66,"Materials (Basel, Switzerland)",0.012132
3,24,Chemical science,0.357263,73,"Molecules (Basel, Switzerland)",0.189108,52,International journal of molecular sciences,0.170636
4,61,Journal of physical therapy science,0.764837,69,Medicine,0.054244,18,BioMed research international,0.028336
5,93,"Sensors (Basel, Switzerland)",0.811183,27,Cureus,0.018530,28,Data in brief,0.014059
6,89,Proceedings of the National Academy of Sciences of the United States of America,0.444906,45,Frontiers in physiology,0.129706,22,Cell reports,0.061258
7,82,PLoS genetics,0.408431,78,Oncogene,0.073535,22,Cell reports,0.057310
8,50,International journal of environmental research and public health,0.677547,89,Proceedings of the National Academy of Sciences of the United States of America,0.058298,93,"Sensors (Basel, Switzerland)",0.035449
9,86,PeerJ,0.252492,91,Royal Society open science,0.173445,89,Proceedings of the National Academy of Sciences of the United States of America,0.101868


In [136]:
# ((y_test==df_max.n1)|(y_test==df_max.n2)).mean()
((y_test==df_max.n1)|(y_test==df_max.n2)|(y_test==df_max.n3)).mean()

0.76683472615512116

1. Some papers clearly fit one journal (high *p*).
2. As expected, some papers are good fits for >1 journals, i.e. similar *p* for >1 journals. 

The model has the right journal in the **top 3** of 105 journals **~77%** of the time for the test data!